In [1]:
import pandas as pd 
import numpy as np
data = pd.read_excel("current_master_6_17_2020.xlsx")
data

,unique_id,source,Census.Year,State/Province,County,Place,Household Joint ID,Joint ID for Matched Records,Last.Name,First.Name,...,Cause of Death,Rank (Military),Enlistment Date,Enlistment Place,Date Mustered Out,Year of this Record,Last Name MATCH,First Name Match,Census Year Match,Total of Matches
0,2388,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,1,1000,101,NaN,NaN,NaN,0,0,0,FALSE
1,2389,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5,5900,1230,NaN,NaN,NaN,0,0,0,FALSE
2,2390,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5,5900,1230,NaN,NaN,NaN,0,0,0,FALSE
3,2391,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5,5900,1230,NaN,NaN,NaN,0,0,0,FALSE
4,2392,1880 IPUMS 100% sample,1880,NaN,NaN,NaN,NaN,NaN,---,---,...,5,5900,1230,NaN,NaN,NaN,0,0,0,FALSE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50372,16981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50373,1892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50374,2044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50375,207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
newdata = data[['unique_id', 'Joint ID for Matched Records', 'Last.Name','First.Name','lat', 'long',\
                'Census.Year', 'State/Province', 'County', 'Place']]

# remove rows with nan or ? in joint id
newdata = newdata.loc[newdata['Joint ID for Matched Records'] != '?'] 
newdata = newdata[pd.notnull(newdata["Joint ID for Matched Records"])]  

# remove rows with nan in lat or long
newdata = newdata[pd.notnull(newdata["lat"])] 
newdata = newdata[pd.notnull(newdata["long"])] 

# remove non digital vals in lat or long
newdata = newdata[pd.to_numeric(newdata['lat'], errors='coerce').notnull()]
newdata = newdata[pd.to_numeric(newdata['long'], errors='coerce').notnull()]

# change census year to int
newdata['Census.Year'] = pd.to_numeric(newdata['Census.Year'], downcast='integer', errors='coerce')

# group by joint id and rank by census year 
newdata['Census.Year'] = newdata['Census.Year'].apply(pd.np.float64)
newdata["rank"] = newdata.groupby("Joint ID for Matched Records")["Census.Year"].rank("dense", ascending=True)

# sort by Joint ID for Matched Records, rank
newdata = newdata.sort_values(by =['Joint ID for Matched Records', 'rank']) 

#newdata.to_csv('tableau_707.csv', index=False)
newdata

,unique_id,Joint ID for Matched Records,Last.Name,First.Name,lat,long,Census.Year,State/Province,County,Place,rank
275,3,1,Addison,James,42.0315,-83.0364,1864.0,CanadaWest,Essex,Windsor,1.0
271,2,1,ADDISON,ALBERTJ,44.7151,-67.4614,1900.0,ME,WASHINGTON,MACHIAS,2.0
47217,5,2,Weeks,Mark,42.0315,-83.0364,1864.0,CanadaWest,Essex,Windsor,1.0
47218,4,2,WEEKS,MARK,42.3314,-83.0458,1880.0,MI,WAYNE,DETROIT,2.0
17466,50863,3,GREEN,CHARITY,42.3583,-71.1096,1860.0,MA,Middlesex,Cambridgeport,1.0
...,...,...,...,...,...,...,...,...,...,...,...
30377,2227,632?,MCNEIL,IRINNE,42.1876,-71.3065,1920.0,MA,NORFOLK,MEDFIELD,2.0
38085,2228,709?,ROSS,ANNIE,41.8781,-87.6298,1900.0,IL,COOK,CHICAGO,1.0
38166,2229,709?,ROSS,SUPERIOR,41.8781,-87.6298,1920.0,IL,COOK,CHICAGO,2.0
23018,2231,970?,Jackson,William,42.0315,-83.0364,1861.0,Ontario,Essex,Windsor,1.0


In [22]:
def angle_between(p1, p2):
    ang1 = np.arctan2(*p1[::-1])
    ang2 = np.arctan2(*p2[::-1])
    return int(np.rad2deg((ang1 - ang2) % (2 * np.pi)))

def angle_list(dataset):
    lat_list = list(dataset.lat)
    long_list = list(dataset.long)
    ans = []
    for i in range(1,len(lat_list)):
        if long_list[i] is None or lat_list[i] is None or \
                long_list[i-1] is None or lat_list[i-1] is None:
            ans.append(0)
        else:
            point = long_list[i]-long_list[i-1], lat_list[i]-lat_list[i-1] 
            ans.append(angle_between(point,[0,0]))
    ans.append(0)
    #for i in range(360):
    #    ans.append(i)
    return ans

def generate_small_df(df):
    cnt_list = df['Joint ID for Matched Records'].value_counts()    
    join_id_list = cnt_list[cnt_list>2].index 
    #join_id_list = [356, 14]
    for join_id in join_id_list:
        #if join_id in [1289,868,108,307,122]: #missing val in lat or long
        #    continue
        #print(join_id)
        match_df = df.loc[df['Joint ID for Matched Records']==join_id].sort_values(by=['Census.Year'])
        original = match_df.reset_index()
        additional = pd.DataFrame({'angle': angle_list(match_df)}).reset_index()
        new = pd.concat([original, additional], axis=1).drop(columns=['index', 'index'])
        #file_name = "tableau/match_file/df" + str(join_id)+".csv"
        #new.to_csv(file_name, index=False)
        new.to_csv('tableau/match_file/df.csv', mode='a', header=False, index=False)

df = pd.DataFrame(columns=['unique_id', 'Joint ID for Matched Records', 'Last.Name', 'First.Name',
       'lat', 'long', 'Census.Year', 'State/Province', 'County', 'Place','rank','angle'])
df.to_csv('tableau/match_file/df.csv', index=False)
generate_small_df(newdata)

In [37]:
newdata.loc[newdata['Joint ID for Matched Records']==356].sort_values(by=['Census.Year'])

,unique_id,Joint ID for Matched Records,Last.Name,First.Name,lat,long,Census.Year,State/Province,County,Place,rank
1148,50814,356,Artis,Elijah,43.6532,-79.3832,1856.0,Canada West,NaN,Toronto,1.0
1172,50822,356,Artiss,Elijah,41.7939,-86.7439,1860.0,MI,Berrien,New Buffalo,2.0
1150,50816,356,Artis,Elijah,41.9861,-82.9264,1861.0,Canada West,Essex,Colchester,3.0
1151,50817,356,Artis,Elijah,42.1321,-83.4252,1870.0,MI,WASHTENAW,Pittsfield,4.0
1144,1866,356,ARTIS,ELI,42.6114,-83.2777,1880.0,MI,OAKLAND,3-WD;PONTIAC,5.0
1152,50818,356,Artis,Elijah,42.1804,-83.745,1880.0,MI,WASHTENAW,Pittsfield,5.0
1153,50819,356,Artis,Elijah,42.2411,-83.613,1880.0,MI,NaN,Ypislanti,5.0
1145,819,356,ARTIS,ELI,42.6389,-83.291,1900.0,MI,OAKLAND,PONTIAC,6.0
1147,820,356,ARTIS,ELIAH,42.6389,-83.291,1910.0,MI,OAKLAND,PONTIAC,7.0
1146,1867,356,ARTIS,ELI,42.6389,-83.291,1920.0,MI,OAKLAND,PONTIAC,8.0


In [38]:
newdata.loc[newdata['Joint ID for Matched Records']==14].sort_values(by=['Census.Year'])

,unique_id,Joint ID for Matched Records,Last.Name,First.Name,lat,long,Census.Year,State/Province,County,Place,rank
3189,50828,14,Bird,John J.,39.1031,-84.512,1850.0,OH,Hamilton,Cincinnati,1.0
3184,30,14,Bird,John,42.0315,-83.0364,1861.0,Ontario,Essex,Windsor,2.0
3185,50826,14,Bird,John,39.3567,-83.3148,1863.0,OH,Ross,Buckskin,3.0
3191,31,14,BIRD,JOHNJ.,37.0053,-89.1765,1880.0,IL,Alexander,Cairo,4.0
3190,50829,14,Bird,John J.,39.7817,-89.6501,1900.0,IL,Sangamon,Springfield,5.0
